# Overview

This notebook uses the 'TextFeaturesGenerator' class (from text_features) to convert textual data into qunatitaive data. 

For now, it creates a bag-of-words representation and a tf-idf representation. We will also add SVD/PCA of these matrices and a Word2Vec representation in the next few days.

Will update the TextFeaturesGenerator class on an ongoing basis and update the usage here.

In [1]:
from text_features import TextFeaturesGenerator
from project_helper import TweetData
import pandas as pd
import numpy as np
from datetime import timedelta  
import datetime

Reusing the TweetData class to get cleaned tweets.

In [2]:
tweet_data = TweetData()
tweet_data.clean_tweets.head()

,tweets
timestamp,
2019-10-02 23:41:51-05:00,democrats want to steal the election
2019-10-02 23:27:52-05:00,mississippi there is a very important election...
2019-10-02 23:27:52-05:00,he loves our military and supports our vets de...
2019-10-02 21:06:36-05:00,look at this photograph
2019-10-02 19:51:56-05:00,schiff house intel chairman got early account ...


Creating a 'TextFeaturesGenerator' instance which takes the tweets as an argument

In [3]:
feature_generator = TextFeaturesGenerator(tweet_data.clean_tweets.tweets)

'get_bow_matrix' creates the bag-of-words matrix

In [4]:
bow_mat = feature_generator.get_bow_matrix()

In [5]:
bow_mat.shape

(27960, 16781)

The shape of this matrix is 27.96K rows (same number as the tweets) and the columns are 16,781, which is equal to the unique number of words in the vocabulary.

In [6]:
bow_mat[:10,:10].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

As you can see, most of the values are zero which is why it is stored as a 'sparse-matrix'

Bag-of-words is simply a count of words in the tweet. A better representation is 'tf-idf'. The 'get_tfidf_matrix' creates

In [7]:
tfidf_mat = feature_generator.get_tfidf_matrix()
tfidf_mat.shape

(27960, 16781)

The matrices can be saved using the matrices function. You can either specify a 'folder' which will be created and both matrices stored in it, else will store in the working directory.

In [8]:
feature_generator.save_matrices()

The two matrices will be saved with the names "bow_mat.npz" and "tfidf_mat.npz"

You can also specify a folder and a suffix to the file names.

In [9]:
feature_generator.save_matrices(folder="matrices",suffix="_v2")

The files can be loaded using the following commands:

In [10]:
from scipy import sparse
bow_loaded = sparse.load_npz("bow_mat.npz")
tfidf_loaded = sparse.load_npz("tfidf_mat.npz")
print(bow_loaded.shape)
print(tfidf_loaded.shape)

(27960, 16781)
(27960, 16781)


## PCA (through SVD) of the matrices

You can get the SVD of the bow and tfidf matrices as well.

In [11]:
svd_bow_mat = feature_generator.get_svd_bow_mat()

In [12]:
svd_bow_mat.shape

(27960, 2)

By default, it gives back two components. You can changet that using the n_components argument.

In [13]:
svd_bow_mat = feature_generator.get_svd_bow_mat(n_components=100)

In [14]:
svd_bow_mat.shape

(27960, 100)

You can get the SVD of the tf-idf as well.

In [15]:
svd_tfidf_mat = feature_generator.get_svd_bow_mat(n_components=100)

In [16]:
svd_tfidf_mat.shape

(27960, 100)

These matrices can be saved as well.

In [17]:
feature_generator.save_matrices()

You can load them back using np.load

In [18]:
svd_loaded_mat = np.load('svd_tfidf_mat.npy')

In [19]:
svd_loaded_mat.shape

(27960, 100)

# Aggregagte SVD per day 

In [20]:
svd_df = pd.DataFrame(svd_loaded_mat)

In [21]:
svd_df['timestamp'] = tweet_data.clean_tweets.index
svd_df['date'] = svd_df.timestamp.dt.date

In [22]:
svd_df.head()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,timestamp,date
0,1.067114,0.077396,-0.803793,-0.205652,-0.505180,0.004843,0.039186,-0.049387,-0.095551,-0.017207,...,-0.038184,0.116668,0.082880,-0.014091,0.017845,0.083295,-0.133397,0.150659,2019-10-02 23:41:51-05:00,2019-10-02
1,2.398382,-2.635966,1.537750,0.957777,0.272032,-0.429117,-1.631507,0.552008,-1.720561,1.136832,...,0.477601,-0.076841,0.183800,0.013123,0.157688,-0.299846,0.255499,-0.004232,2019-10-02 23:27:52-05:00,2019-10-02
2,2.039926,-2.169685,2.635236,0.428123,-0.238948,-0.022691,-0.655132,-0.462280,-1.005658,1.190836,...,-0.218651,-0.002521,0.446379,-0.422941,0.125603,-0.709889,0.274716,-0.015907,2019-10-02 23:27:52-05:00,2019-10-02
3,0.116174,-0.057314,0.003153,0.019891,0.105173,-0.114806,-0.126431,-0.100204,-0.022856,-0.030552,...,-0.022863,0.074032,0.009432,-0.027872,0.046263,-0.048488,-0.025964,-0.056114,2019-10-02 21:06:36-05:00,2019-10-02
4,0.473519,-0.025296,0.216236,0.710462,0.506245,0.856471,-0.055029,-0.015950,-0.015656,-0.262653,...,0.125645,0.155059,-0.043188,0.023508,-0.021414,-0.080335,-0.030428,-0.232628,2019-10-02 19:51:56-05:00,2019-10-02


In [23]:
svd_df_daily = svd_df.groupby('date').agg(np.mean)

In [24]:
svd_df_daily

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
date,,,,,,,,,,,,,,,,,,,,,
2009-05-04,1.917882,-0.737776,-0.008240,-0.307825,0.125388,-0.765858,0.104030,-0.852788,-0.843460,-0.306545,...,-0.093716,-0.001603,-0.175242,-0.210991,-0.030184,0.047171,0.120859,0.238349,0.044229,-0.099610
2009-05-05,1.734408,-0.729916,-0.037521,-0.528596,-0.111331,-0.610496,-0.951095,-0.822610,-0.527629,-0.409863,...,-0.143628,-0.031154,-0.015099,-0.109080,0.123480,-0.065329,0.089585,0.072774,0.127022,-0.087434
2009-05-08,0.656465,0.025012,0.336565,-0.142836,-0.174618,-0.143168,-0.148943,-0.144386,-0.311437,-0.002341,...,-0.092005,0.005041,-0.009977,-0.051010,0.034604,-0.034992,-0.064200,0.081428,-0.047919,0.084197
2009-05-12,0.764610,-0.615122,-0.252076,-0.132610,0.900807,-0.321875,-0.307898,-0.607115,0.308672,-0.375667,...,-0.074368,-0.033476,-0.073585,-0.018872,-0.000607,-0.174409,0.027882,0.002136,0.118771,0.013529
2009-05-13,0.552576,-0.718454,-0.642428,-0.030314,-0.052141,0.129544,0.069838,-0.090266,-0.274465,-0.070405,...,-0.111563,-0.070154,0.049840,-0.132478,0.066403,-0.067258,0.009621,0.102076,0.104590,-0.027288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-28,1.766456,-0.018465,-0.006562,-0.071148,-0.085782,-0.067368,0.134505,0.280319,-0.044777,0.094144,...,-0.042519,0.011280,0.123924,-0.040401,-0.055683,-0.034498,0.049269,-0.078064,0.048818,-0.006177
2019-09-29,1.818351,0.067952,-0.139502,-0.020150,-0.080780,-0.107743,0.294175,-0.018606,0.171831,0.062312,...,-0.014597,0.063705,-0.007438,-0.025048,0.030614,0.026506,-0.009292,0.039754,0.029842,-0.031893
2019-09-30,2.838771,0.208005,0.114255,0.087938,-0.168927,-0.027974,0.171122,0.041970,0.033269,-0.042353,...,0.011057,0.027501,0.047761,-0.043513,0.023306,0.026424,0.000217,0.049866,0.012281,-0.017940


In [25]:
svd_df_daily.to_csv('svd_df_daily.csv')

# 4 PM

In [26]:
tweet_data.clean_tweets['timestamp'] = tweet_data.clean_tweets.index
after_4_tweets = tweet_data.clean_tweets.timestamp.dt.hour >= 15
tweet_data.clean_tweets['after4_date'] = tweet_data.clean_tweets.timestamp.dt.date
tweet_data.clean_tweets.loc[after_4_tweets,'after4_date'] = tweet_data.clean_tweets.timestamp[after_4_tweets].dt.date + timedelta(days=1)

In [27]:
tweet_data.clean_tweets.head(100)

,tweets,timestamp,after4_date
timestamp,,,
2019-10-02 23:41:51-05:00,democrats want to steal the election,2019-10-02 23:41:51-05:00,2019-10-03
2019-10-02 23:27:52-05:00,mississippi there is a very important election...,2019-10-02 23:27:52-05:00,2019-10-03
2019-10-02 23:27:52-05:00,he loves our military and supports our vets de...,2019-10-02 23:27:52-05:00,2019-10-03
2019-10-02 21:06:36-05:00,look at this photograph,2019-10-02 21:06:36-05:00,2019-10-03
2019-10-02 19:51:56-05:00,schiff house intel chairman got early account ...,2019-10-02 19:51:56-05:00,2019-10-03
...,...,...,...
2019-09-28 03:55:17-05:00,thank you to general mcmaster just more fake n...,2019-09-28 03:55:17-05:00,2019-09-28
2019-09-28 02:03:00-05:00,,2019-09-28 02:03:00-05:00,2019-09-28
2019-09-27 19:41:18-05:00,i am draining the swamp,2019-09-27 19:41:18-05:00,2019-09-28


In [28]:
combined_daily_tweets = tweet_data.clean_tweets.groupby('after4_date')['tweets'].apply(lambda x: ' '.join(x))
combined_daily_tweets.head()

after4_date
2009-05-05    donald trump will be appearing on the view tom...
2009-05-08    donald trump reads top ten financial tips on l...
2009-05-09    new blog post celebrity apprentice finale and ...
2009-05-12    my persona will never be that of a wallflower ...
2009-05-13    miss usa tara conner will not be fired ive alw...
Name: tweets, dtype: object

In [29]:
combined_daily_tweets.to_csv('combined_daily_tweets.csv')

c:\users\gufra\onedrive\documents\academics\introtodatascience\final_project\trump_tweets\venv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


# Check if the concatenation is correct

In [30]:
tweet_data.clean_tweets.tweets[tweet_data.clean_tweets.after4_date==pd.to_datetime("2019-10-03")]

timestamp
2019-10-02 23:41:51-05:00                democrats want to steal the election 
2019-10-02 23:27:52-05:00    mississippi there is a very important election...
2019-10-02 23:27:52-05:00    he loves our military and supports our vets de...
2019-10-02 21:06:36-05:00                             look at this photograph 
2019-10-02 19:51:56-05:00    schiff house intel chairman got early account ...
2019-10-02 15:48:47-05:00    the do nothing democrats should be focused on ...
2019-10-02 15:39:07-05:00    adam schiff should only be so lucky to have th...
2019-10-02 15:31:53-05:00    democrats are trying to undo the election rega...
2019-10-02 15:31:03-05:00    nancy pelosi just said that she is interested ...
2019-10-02 15:19:09-05:00    all of this impeachment nonsense which is goin...
2019-10-02 15:02:11-05:00    now the press is trying to sell the fact that ...
Name: tweets, dtype: object

In [31]:
combined_daily_tweets[combined_daily_tweets.index.values==pd.to_datetime("2019-10-03")]

after4_date
2019-10-03    democrats want to steal the election  mississi...
Name: tweets, dtype: object

# Create SVD matrix of the combiened 4 PM tweets

In [32]:
combined_generator = TextFeaturesGenerator(combined_daily_tweets)

In [40]:
n_components = 2
combined_svd_df = pd.DataFrame(combined_generator.get_svd_tfidf_mat(n_components=n_components))

In [41]:
combined_svd_df['after4_date'] = combined_daily_tweets.index.values

In [42]:
combined_svd_df.head()

,0,1,after4_date
0,0.231443,0.194505,2009-05-05
1,0.052787,0.062026,2009-05-08
2,0.079677,0.035517,2009-05-09
3,0.102123,0.042712,2009-05-12
4,0.068873,0.061913,2009-05-13


In [43]:
combined_svd_df.to_csv('combined_svd_df.csv')